In [1]:
## importing libraries

#!pip install tensorflow
from glob import glob
import guitarpro
from guitarpro import *
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import pickle
from tqdm import tqdm
import gzip
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten


2024-04-30 21:29:03.248306: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('Jazz-midi.csv')

missing_values = df.isnull().sum()
duplicates = df.duplicated().sum()

df = df.drop(columns=['Unnamed: 0'])
df['Name'] = df['Name'].str.replace(r'.mid', '', regex=True)

new_df = df.sort_values(by='Name', ascending=True)

new_df.set_index('Name', inplace=True)

In [3]:
new_df

,Notes,Len_Sequence,Unique_notes,len_Uni_Notes
Name,,,,
2ndMovementOfSinisterFootwear,"['C#4', 'E4', 'G4', 'E-5', 'B-4', 'C5', 'C#4',...",2027,"{'A2', '9.2', 'C#6', 'B4', 'A3', 'D5', 'E3', '...",49
55Dive,"['C#2', 'C#2', 'E2', 'E2', 'E2', 'E2', 'E2', '...",664,"{'3.4', 'A2', '11.2.5', '0.4.7', '6.8.0', 'A3'...",47
5To10,['9.0'],1,{'9.0'},1
634-5789,"['D5', 'F#5', 'A5', 'F#5', 'E5', 'D5', 'B4', '...",264,"{'F#5', 'A4', 'F5', 'G5', 'B-5', 'C5', 'A5', '...",11
914,"['E1', 'E1', 'E1', 'E1', 'E1', 'E1', 'E1', 'E1...",1763,"{'E6', '3.4', 'A6', '9', 'A2', '6', '9.2', 'C#...",70
...,...,...,...,...
YouveChanged,"['E-2', 'B-2', 'E-2', 'A1', 'D2', 'G#2', 'C#2'...",439,"{'A0', 'C3', 'B-0', 'F1', 'A2', 'E-1', 'B1', '...",28
YouveMadeMeSoVeryHappy,"['G2', 'G3', 'G#2', '7', '10.1.3.6', '9.0.2.5'...",214,"{'7.10.2', '1', '0.2.6.8', '9', '6', '0.4.7', ...",50
ZootAllures,"['A1', 'G#3', 'A1', 'C#4', 'E-4', 'E4', 'E-4',...",462,"{'2.5', '6.8', '3.4', '8.9.11.1.4', '6.8.11', ...",64


In [4]:
import shutil

loc = '/Volumes/Macintosh HD/Ashhwath/Masters/GWU/Classes/Spring 2024/Capstone project/Data/archive/Jazz Midi'

midi_files = [f for f in os.listdir(loc) if f.endswith('.mid')]

new_loc = '/Volumes/Macintosh HD/Ashhwath/Masters/GWU/Classes/Spring 2024/Capstone project/Data/archive/Jazz Midi New'
os.makedirs(loc, exist_ok=True)

for midi_file in midi_files:
    name_without_extension = os.path.splitext(midi_file)[0]
    if name_without_extension in new_df.index:
        source_path = os.path.join(loc, midi_file)
        destination_path = os.path.join(new_loc, midi_file)
        shutil.copy(source_path, destination_path)
        print(f"Copied: {midi_file}")

#f"Files in new directory: {
os.listdir(new_loc)

Copied: GlasgowKiss.mid
Copied: Swear.mid
Copied: HowDoYouKeepTheMusicPlaying.mid
Copied: MessageOfLove.mid
Copied: Spiderman.mid
Copied: KnocksMeOffMyFeet.mid
Copied: Fallen.mid
Copied: JungiNishan.mid
Copied: Alone.mid
Copied: Tracie.mid
Copied: AWomansWorth.mid
Copied: BetterDaysAhead.mid
Copied: MacktheKnife.mid
Copied: Chinatawn.mid
Copied: IllSaySheDoes.mid
Copied: LoveIsTheSeventhWave.mid
Copied: WhosSorryNow.mid
Copied: YoungAtHeart.mid
Copied: YoureGonnaBeSorry.mid
Copied: DamageControl.mid
Copied: WatermelonMan.mid
Copied: LilyWasHere.mid
Copied: BeneathADesertMoon.mid
Copied: WednesdaysChild.mid
Copied: SummerWine.mid
Copied: AllBlues.mid
Copied: IfIHadYou.mid
Copied: WayYouLookTonight.mid
Copied: ChildrenSay.mid
Copied: Lakes.mid
Copied: ForHeavensSake.mid
Copied: StTropez.mid
Copied: Youllneverfind.mid
Copied: Rawhide.mid
Copied: SoulMates.mid
Copied: EtudeInAMinorOpus10No2.mid
Copied: BitterMoon.mid
Copied: Rumble.mid
Copied: 634-5789.mid
Copied: ImSoHappyICantStopCrying.

['GlasgowKiss.mid',
 'Swear.mid',
 'HowDoYouKeepTheMusicPlaying.mid',
 'MessageOfLove.mid',
 'Spiderman.mid',
 'KnocksMeOffMyFeet.mid',
 'Fallen.mid',
 'JungiNishan.mid',
 'Alone.mid',
 'Tracie.mid',
 'AWomansWorth.mid',
 'BetterDaysAhead.mid',
 'MacktheKnife.mid',
 'Chinatawn.mid',
 'IllSaySheDoes.mid',
 'LoveIsTheSeventhWave.mid',
 'WhosSorryNow.mid',
 'YoungAtHeart.mid',
 'YoureGonnaBeSorry.mid',
 'DamageControl.mid',
 'WatermelonMan.mid',
 'LilyWasHere.mid',
 'BeneathADesertMoon.mid',
 'WednesdaysChild.mid',
 'SummerWine.mid',
 'AllBlues.mid',
 'IfIHadYou.mid',
 'WayYouLookTonight.mid',
 'ChildrenSay.mid',
 'Lakes.mid',
 'ForHeavensSake.mid',
 'StTropez.mid',
 'Youllneverfind.mid',
 'Rawhide.mid',
 'SoulMates.mid',
 'EtudeInAMinorOpus10No2.mid',
 'BitterMoon.mid',
 'Rumble.mid',
 '634-5789.mid',
 'ImSoHappyICantStopCrying.mid',
 'SomethingTheBoySaid.mid',
 'CleanSweep.mid',
 'BuglersDream.mid',
 'Whispering.mid',
 'ThisCowboySong.mid',
 'AngelEyes.mid',
 'TigerRag.mid',
 'ItWasAVer

In [5]:
filtered_df = new_df[new_df.index.isin([os.path.splitext(f)[0] for f in os.listdir(new_loc)])]
filtered_df.to_csv('filtered_dataset.csv')

filtered_df

,Notes,Len_Sequence,Unique_notes,len_Uni_Notes
Name,,,,
2ndMovementOfSinisterFootwear,"['C#4', 'E4', 'G4', 'E-5', 'B-4', 'C5', 'C#4',...",2027,"{'A2', '9.2', 'C#6', 'B4', 'A3', 'D5', 'E3', '...",49
55Dive,"['C#2', 'C#2', 'E2', 'E2', 'E2', 'E2', 'E2', '...",664,"{'3.4', 'A2', '11.2.5', '0.4.7', '6.8.0', 'A3'...",47
5To10,['9.0'],1,{'9.0'},1
634-5789,"['D5', 'F#5', 'A5', 'F#5', 'E5', 'D5', 'B4', '...",264,"{'F#5', 'A4', 'F5', 'G5', 'B-5', 'C5', 'A5', '...",11
914,"['E1', 'E1', 'E1', 'E1', 'E1', 'E1', 'E1', 'E1...",1763,"{'E6', '3.4', 'A6', '9', 'A2', '6', '9.2', 'C#...",70
...,...,...,...,...
YouveChanged,"['E-2', 'B-2', 'E-2', 'A1', 'D2', 'G#2', 'C#2'...",439,"{'A0', 'C3', 'B-0', 'F1', 'A2', 'E-1', 'B1', '...",28
YouveMadeMeSoVeryHappy,"['G2', 'G3', 'G#2', '7', '10.1.3.6', '9.0.2.5'...",214,"{'7.10.2', '1', '0.2.6.8', '9', '6', '0.4.7', ...",50
ZootAllures,"['A1', 'G#3', 'A1', 'C#4', 'E-4', 'E4', 'E-4',...",462,"{'2.5', '6.8', '3.4', '8.9.11.1.4', '6.8.11', ...",64


In [6]:
import pretty_midi
import os

def extract_midi_features(midi_file_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_file_path)
        notes = []
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                notes.append({
                    'pitch': note.pitch,
                    'start': note.start,
                    'end': note.end,
                    'velocity': note.velocity,
                    'instrument': instrument.program
                })
        return notes
    except Exception as e:
        f"Error processing {midi_file_path}: {e}"
        return None


In [7]:
def normalize_notes(notes):
    """ Normalize pitches and times to a more uniform range and scale. """
    # Example: Transpose notes to C major/A minor
    normalized_notes = []
    for note in notes:
        normalized_pitch = (note['pitch'] % 12)  # Transpose pitches within an octave
        normalized_notes.append({
            'pitch': normalized_pitch,
            'start': round(note['start'], 2),  # Round off start times
            'end': round(note['end'], 2),
            'velocity': note['velocity']
        })
    return normalized_notes

In [8]:
def process_midi_directory(directory):
    all_midi_data = []
    for filename in os.listdir(directory):
        if filename.endswith('.mid'):
            file_path = os.path.join(directory, filename)
            features = extract_midi_features(file_path)
            if features:
                normalized_features = normalize_notes(features)
                all_midi_data.append({
                    'filename': filename,
                    'features': normalized_features
                })
    return all_midi_data


In [ ]:
import json

#output file = "/Volumes/Macintosh HD/Ashhwath/Masters/GWU/Classes/Spring 2024/Capstone project/Data/archive/Jazz Midi normal"

def save_processed_data(data, output_file):
    with open(output_file, 'w') as f:
        json.dump(data, f)

# Use the function
midi_directory = '/Volumes/Macintosh HD/Ashhwath/Masters/GWU/Classes/Spring 2024/Capstone project/Data/archive/Jazz Midi New'
processed_data = process_midi_directory(midi_directory)
save_processed_data(processed_data, 'processed_midi_data.json')


In [20]:
print("Current working directory:", os.getcwd())

Current working directory: /Users/Ashhrock777


In [12]:
#song1 = 'TaketheAtrain'

#if song1 in new_df['Name'].values:
#    print(f"{song1} is in the column.")
#else:
#    print(f"{song1} is not in the column.")